[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ds-reboot/python-first-part/blob/main/notebooks/hometasks/Task3_dmrf_OOP.ipynb)

# Задание
Реализовать с помощью объектно-ориентированного подхода предыдущие 2 задания. Создайте для каждой из задач отдельный класс, который позволяет ее решить.

* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe, а также в excel, pickle, БД

* Проверить состояние датафрейма и привести его в формат, позволяющий дальнейшее исследование данных

*  Сделать визуализацию для мини-исследования рынка строящейся недвижимости в одном или нескольких регионах с помощью pandas, matplotlib, seaborn, plotly и других инструментов.



In [83]:
import requests
from tqdm.auto import tqdm
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns

In [84]:
class DomIdLoader:
    def __init__(self):
        self.offset_ = 1
        self.limit_ = 1000
        self.url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object'
        self.obj_ids = []

    def get_ids(self):
        while True:
            paramz = {
                'offset': self.offset_,
                'limit': self.limit_,
                'sortField':'devId.devShortCleanNm',
                'sortType':'asc',
                'objStatus':'0',
            }
            res = requests.get(self.url, params=paramz).json()
            objects_list = res.get('data').get('list')
            
            print(self.offset_, self.limit_, len(self.obj_ids), len(objects_list))
            
            self.obj_ids = self.obj_ids + [x.get('objId') for x in objects_list]
            if len(objects_list) < self.limit_:
                break
            self.offset_ += self.limit_
               
        return self.obj_ids

    def show_ids(self):
        print(self.obj_ids)

In [85]:
id_loader = DomIdLoader()
ids = id_loader.get_ids()
len(ids)

1 1000 0 1000
1001 1000 1000 1000
2001 1000 2000 1000
3001 1000 3000 1000
4001 1000 4000 1000
5001 1000 5000 1000
6001 1000 6000 1000
7001 1000 7000 1000
8001 1000 8000 1000
9001 1000 9000 1000
10001 1000 10000 310


10310

In [86]:
class ObjectInfoExtractor:
    def __init__(self, ids):
        self.ids = ids
        self.url = 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/'
        self.data = []
        
    def get_object(self, idx):
        url = self.url + f'{idx}'
        res = requests.get(url).json().get('data')
        return res
    
    def load_data(self):
        for idx in tqdm(self.ids):
            self.data = self.data + [self.get_object(idx)]
        return self.data

    def df_converter(self):
        df = pd.json_normalize(self.data)
        return df

In [87]:
oie = ObjectInfoExtractor(ids)
data = oie.load_data()

  0%|          | 0/10310 [00:00<?, ?it/s]

In [88]:
df = oie.df_converter()

In [89]:
df.head(2)

,id,hobjId,pdId,region,address,floorMin,floorMax,objElemLivingCnt,objReady100PercDt,wallMaterialShortDesc,...,metro.time,metro.isWalk,metro.colors,airQualityIndexValue,developer.bankruptStage.bankruptStageCd,developer.bankruptStage.bankruptStageDesc,developer.bankruptStage.bankruptLawUrl,developer.orgBankruptMsgDttm,complexShortNm,greenAreaIndexValue
0,13383,39024,28105,72,г Тюмень,4,20,68,2023-03-31,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13398,37165,28141,72,"г Тюмень, ул Профсоюзная, д. 56",7,10,43,2024-12-31,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
class Saver:
    def __init__(self, data):
        self.df = data

    def save_csv(self):
        self.df.to_csv('data.csv')

    def save_xl(self):
        self.df.to_excel('data.xlsx')

    def save_sql(self):
        engine = create_engine('sqlite://', echo=False)
        with engine.begin() as connection:
            self.df.to_sql('df', con=connection, if_exists='replace', index=False)
        return engine

In [91]:
saver = Saver(df)
saver.save_csv()
saver.save_xl()

In [92]:
class Visualizer:
    def __init__(self, data):
        self.df = data

    def make_boxplot(self):
        sns.set(style='darkgrid')
        sns.catplot(
            x='objLkClassDesc', 
            y='objPriceAvg', 
            data=self.df, 
            kind='box', 
            height=4, 
            aspect=2
        );

In [93]:
vis = Visualizer(df)

In [94]:
vis.make_boxplot()